## 3. 无重复字符的最长子串

滑窗 + 哈希表

In [37]:
from collections import defaultdict
class Solution:
    def lengthOfLongestSubstring(self, s: str) -> int:
        cnt = defaultdict(int)
        left = right = 0
        ans = 0
        while right < len(s):
            # print(cnt)
            cur = s[right]
            cnt[cur] += 1
            while cnt[cur] > 1 and left <= right:
                cnt[s[left]] -= 1
                left += 1
            right += 1
            ans = max(ans, right - left)
        return ans

In [38]:
s = ""

In [39]:
S = Solution()
S.lengthOfLongestSubstring(s)

0

也可以用set代替哈希表的作用

## 206. 反转链表

链表常见双指针

In [40]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

In [43]:
from typing import Optional
class Solution:
    def reverseList(self, head: Optional[ListNode]) -> Optional[ListNode]:
        pre, cur = None, head
        while cur:
            nxt = cur.next
            cur.next = pre
            pre = cur
            cur = nxt
        return pre

### 递归

In [45]:
class Solution:
    def reverseList(self, head: Optional[ListNode]) -> Optional[ListNode]:
        def reverse(pre, cur):
            if not cur: return pre
            nxt = cur.next
            cur.next = pre
            return reverse(cur, nxt)
        return reverse(None, head)

## 146. LRU缓存 (重要)

双向链表 + 哈希表

哈希表中存放双向链表的节点，这样就能以 O(1) 的复杂度进行查询；定位到要操作的节点之后，双链表的增删改都是 O(1) 的复杂度。 同时双向链表中存储键值对，查询映射关系。

In [9]:
class DLinkedNode:
    """双向链表"""
    def __init__(self, key = 0, value = 0):
        self.key = key
        self.value = value
        self.prev = None
        self.next = None

class LRUCache:

    def __init__(self, capacity: int):
        self.head = DLinkedNode()
        self.tail = DLinkedNode()
        self.head.next = self.tail
        self.tail.prev = self.head
        self.capacity = capacity
        self.size = 0
        self.cache = dict()
        
    def get(self, key: int) -> int:
        if key not in self.cache:
            return -1
        node = self.cache[key]
        self.moveToHead(node)
        return node.value
        
    def put(self, key: int, value: int) -> None:
        if key not in self.cache:
            node = DLinkedNode(key, value)
            self.cache[key] = node
            self.addToHead(node)
            self.size += 1
            if self.size > self.capacity:
                removed = self.removeTail()
                self.cache.pop(removed.key)
                self.size -= 1
        else:
            node = self.cache[key]
            node.value = value
            self.moveToHead(node)
    
    def addToHead(self, node):
        node.prev = self.head
        node.next = self.head.next
        self.head.next.prev = node
        self.head.next = node
        
    def removeNode(self, node):
        node.next.prev = node.prev
        node.prev.next = node.next
    
    def moveToHead(self, node):
        self.removeNode(node)
        self.addToHead(node)
        
    def removeTail(self):
        node = self.tail.prev
        self.removeNode(node)
        return node
        
        

## 215. 数组中的第K个最大元素

### 快速选择

In [13]:
import random
from typing import List
class Solution:
    def findKthLargest(self, nums: List[int], k: int) -> int:
        def partition(nums, l, r):
            cur = random.randint(l, r)
            x, i = nums[cur], l
            nums[cur], nums[r] = nums[r], nums[cur]
            for j in range(l, r):
                if nums[j] <= x:
                    nums[j], nums[i] = nums[i], nums[j]
                    i += 1
            nums[r], nums[i] = nums[i], nums[r]
            return i
        
        def quick_select(nums, l, r, k):
            idx = partition(nums, l, r)
            if idx == k: return nums[idx]
            return quick_select(nums, idx + 1, r, k) if idx < k else\
            quick_select(nums, l, idx - 1, k)
        
        return quick_select(nums, 0, len(nums) - 1, len(nums) - k)

### 最小堆

In [16]:
import heapq
class Solution:
    def findKthLargest(self, nums: List[int], k: int) -> int:
        heap = []
        for n in nums:
            if len(heap) == k: heapq.heappushpop(heap, n)
            else: heapq.heappush(heap, n)
        return heap[0]

## 25. K个一组反转链表

重点在于细节的考虑

In [1]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

In [3]:
class Solution:
    def reverse(self, head: ListNode, tail: ListNode):
        """翻转一个子链表，并且返回新的头与尾"""
        pre = tail.next
        cur = head
        while pre != tail:
            nxt = cur.next
            cur.next = pre
            pre = cur
            cur = nxt
        return tail, head
    
    def reverseKGroup(self, head: ListNode, k: int) -> ListNode:
        dumb = ListNode(0, head)
        pre = dumb
    
        while head:
            tail = pre
            # 判断剩余部分长度是否大于等于 k
            # 大于的话定位到 要翻转的最后一个节点，否则之间返回链表的头节点
            for _ in range(k):
                tail = tail.next
                if not tail:
                    return dumb.next
            nxt = tail.next
            head, tail = self.reverse(head, tail)
            # 把子链表重新接回原链表
            pre.next = head
            tail.next = nxt
            pre = tail
            head = tail.next
            
        return dumb.next

## 15. 三数之和

要点：在第二层遍历中使用双指针反向查询目标和

In [8]:
from typing import List
class Solution:
    def threeSum(self, nums: List[int]) -> List[List[int]]:
        nums.sort()
        n, res = len(nums), []
        for i in range(n):
            if i > 0 and nums[i] == nums[i - 1]: continue
            tgt = -nums[i]
            k = n - 1  # 每次都重新枚举一遍 k
            for j in range(i + 1, n):
                if j > i + 1 and nums[j] == nums[j - 1]: continue
                while j < k + 1 and nums[j] + nums[k] > tgt: k -= 1
                if j == k: break  # 指针重合时特判，直接 break
                if nums[j] + nums[k] == tgt:
                    res.append([nums[i], nums[j], nums[k]])
        return res

## 53. 最大子数组和

### 动态规划

思路：考虑以 nums[i] 为结尾的最大子数组和，则 nums[i] 可以单独成段或与前面的数字一起，即：

dp[i] = max(nums[i], dp[i - 1] + nums[i])， dp[-1] = -∞，遍历时维护最大值即可。

In [9]:
class Solution:
    def maxSubArray(self, nums: List[int]) -> int:
        ans = nums[0]
        pre = -float("Inf")
        for n in nums:
            pre = n if pre < 0 else pre + n
            ans = max(ans, pre)
        return ans

## 912. 排序数组

In [48]:
from random import randint
from typing import List
class Solution:
    def sortArray(self, nums: List[int]) -> List[int]:
        def quick_sort(l, r):
            if l < r:
                x, cur = randint(l, r), l
                nums[x], nums[r] = nums[r], nums[x]
                for i in range(l, r):
                    if nums[i] < nums[r]:
                        nums[cur], nums[i] = nums[i], nums[cur]
                        cur += 1
                nums[cur], nums[r] = nums[r], nums[cur]
                quick_sort(l, cur - 1)
                quick_sort(cur + 1, r)
        quick_sort(0, len(nums) - 1)
        return nums

In [27]:
nums = [5,1,1,2,0,0]

In [28]:
s = Solution()
s.sortArray(nums)

[0, 0, 1, 1, 2, 5]

原始的快排过不了最后一个重复数字很多的例子，需用到三路快排

### 三路快排

将排序的数据分为三部分，小于 x 等于 x 和大于 x，这样只需对前后两部分进行排序即可，用于解决重复数字较多的情况。

In [26]:
from random import randint
from typing import List
class Solution:
    def sortArray(self, nums: List[int]) -> List[int]:
        def quick_sort(arr, l, r):
            if l >= r:
                return
            random_index = random.randint(l, r)
            pivot = arr[random_index]
            arr[l], arr[random_index] = arr[random_index], arr[l]
            i = l + 1
            j = l 
            k = r + 1
            while i < k:
                if arr[i] < pivot:
                    arr[i], arr[j + 1] = arr[j + 1], arr[i]
                    j += 1
                    i += 1
                elif arr[i] > pivot:
                    arr[i], arr[k - 1] = arr[k - 1], arr[i]
                    k -= 1
                else: 
                    i += 1
            arr[l], arr[j] = arr[j], arr[l]
            quick_sort(arr, l, j - 1)
            quick_sort(arr, k, r)
                
        quick_sort(nums, 0, len(nums) - 1)
        return nums

In [7]:
import random
def quick_sort(arr, l, r):
    if l >= r:
        return
    random_index = random.randint(l, r)
    pivot = arr[random_index]
    arr[l], arr[random_index] = arr[random_index], arr[l]
    i = l + 1
    j = l 
    k = r + 1
    while i < k:
        if arr[i] < pivot:
            arr[i], arr[j + 1] = arr[j + 1], arr[i]
            j += 1
            i += 1
        elif arr[i] > pivot:
            arr[i], arr[k - 1] = arr[k - 1], arr[i]
            k -= 1
        else: 
            i += 1
    arr[l], arr[j] = arr[j], arr[l]
    quick_sort(arr, l, j - 1)
    quick_sort(arr, k, r)

In [19]:
arr = [5,4,3,2,1,0]
quick_sort(arr, 0, 5)
arr

[0, 1, 2, 3, 4, 5]

## 21. 合并两个有序链表

In [1]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

### 迭代

In [3]:
from typing import Optional
class Solution:
    def mergeTwoLists(self, list1: Optional[ListNode], list2: Optional[ListNode]) -> Optional[ListNode]:
        dumb = ListNode()
        cur = dumb
        while list1 and list2:
            if list1.val <= list2.val:
                cur.next = list1
                list1 = list1.next
            else:
                cur.next = list2
                list2 = list2.next
            cur = cur.next
        if list1: cur.next = list1
        if list2: cur.next = list2
        return dumb.next

### 递归

In [6]:
from typing import Optional
class Solution:
    def mergeTwoLists(self, list1: Optional[ListNode], list2: Optional[ListNode]) -> Optional[ListNode]:
        if not list1: return list2
        if not list2: return list1
        if list1.val <= list2.val:
            list1.next = self.mergeTwoLists(list1.next, list2)
        else: list2.next = self.mergeTwoLists(list1, list2.next)
        return list1 if list1.val <= list2.val else list2

## 20. 有效的括号

In [10]:
class Solution:
    def isValid(self, s: str) -> bool:
        tab = {")": "(", "]": "[", "}": "{"}
        stk = []
        for c in s:
            if c in tab:
                if not stk or tab[c] != stk.pop(): 
                    return False
            else: stk.append(c)
        return not stk

## 121. 买卖股票的最佳时机

In [12]:
from typing import List
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        min_ = float("inf")
        ans = 0
        for n in prices:
            min_ = min(min_, n)
            ans = max(ans, n - min_)
        return ans

## 141. 环形链表

In [13]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, x):
        self.val = x
        self.next = None

In [16]:
class Solution:
    def hasCycle(self, head: Optional[ListNode]) -> bool:
        slow = fast = head
        while fast and fast.next and fast.next.next:
            slow = slow.next
            fast = fast.next.next
            if slow == fast:
                return True
        return False

## 5. 最长回文串

In [18]:
class Solution:
    def longestPalindrome(self, s: str) -> str:
        def expand(l, r):
            while l >= 0 and r < n and s[l] == s[r]:
                l -= 1
                r += 1
            return l + 1, r  # 返回的值刚好可以作为索引
        
        n = len(s)
        left = right = 0
        for i in range(n):
            l1, r1 = expand(i, i + 1)
            l2, r2 = expand(i, i)
            if r1 - l1 > right - left: left, right = l1, r1
            if r2 - l2 > right - left: left, right = l2, r2
            
        return s[left: right]

## 236. 二叉树的最近公共祖先

In [19]:
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, x):
        self.val = x
        self.left = None
        self.right = None

In [29]:
class Solution:
    def lowestCommonAncestor(self, root: 'TreeNode', p: 'TreeNode', q: 'TreeNode') -> 'TreeNode':
        def dfs(node, path):
            if node == p or node == q:
                res.append(path)
            if node.left:
                dfs(node.left, path + [node.left])
            if node.right:
                dfs(node.right, path + [node.right])
        res = []
        dfs(root, [root])
        res.sort(key = lambda x: len(x))
        # print(res)
        for node in res[1]:
            if node in res[0]:
                ans = node
        return ans

### 存储父节点

In [38]:
class Solution:
    def lowestCommonAncestor(self, root: 'TreeNode', p: 'TreeNode', q: 'TreeNode') -> 'TreeNode':
        def dfs(node, fa):
            cnt[node] = fa
            if node.left:
                dfs(node.left, node)
            if node.right:
                dfs(node.right, node)
        
        cnt = {}
        dfs(root, None)
        vis = set()
        cur = p
        while cur:
            vis.add(cur)
            cur = cnt[cur]
        cur = q
        while cur:
            if cur in vis: return cur
            cur = cnt[cur]
        return 

### 巧妙的递归

> 我们递归遍历整棵二叉树，定义 $f_x$ 表示 $x$ 节点的子树中是否包含 $p$ 节点或 $q$ 节点，如果包含为 $true$，否则为 $false$。那么符合条件的最近公共祖先 $x$ 一定满足如下条件：
$$(f_{lson}\ \&\&\ f_{rson})\ ||\ (x = p || x = q)\ \&\&\ (f_{lson}\ ||\ f_{rson})$$

In [40]:
class Solution:
    def lowestCommonAncestor(self, root: 'TreeNode', p: 'TreeNode', q: 'TreeNode') -> 'TreeNode':
        def dfs(x, p, q):
            nonlocal ans
            if not x: return False
            lson = dfs(x.left, p, q)
            rson = dfs(x.right, p, q)
            # 满足LCA节点的条件
            if (lson and rson) or (x.val == p.val or x.val == q.val) and (lson or rson):
                ans = x
            # 每个节点应当返回的值
            return lson or rson or x.val == p.val or x.val == q.val
        ans = None
        dfs(root, p, q)
        return ans

## 54. 螺旋矩阵

nb 做法

In [6]:
from typing import List
class Solution:
    def spiralOrder(self, matrix: List[List[int]]) -> List[int]:
        res = []
        while matrix:
            # 削头
            res += matrix.pop(0)
            # 顺时针旋转 90 度再削头
            # list(zip(*matrix)) 的作用为将矩阵逆时针旋转 90 度
            # [::-1]再翻转，相当于顺时针 90 度
            matrix = list(zip(*matrix))[::-1]
        return res

## 23. 合并 k 个升序链表 

In [16]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

In [19]:
from collections import deque
from typing import Optional

class Solution:
    
    def merge(self, l1, l2):
        if not l1: return l2
        if not l2: return l1
        if l1.val < l2.val: l1.next = self.merge(l1.next, l2)
        else: l2.next = self.merge(l1, l2.next)
        return l1 if l1.val < l2.val else l2
        
    def mergeKLists(self, lists: List[Optional[ListNode]]) -> Optional[ListNode]:
        dumb = ListNode()
        lists = deque(lists)
        while lists:
            cur1 = lists.pop()
            if not lists: return cur1
            cur2 = lists.pop()
            lists.append(self.merge(cur1, cur2))
        return

## 92. 反转链表Ⅱ

In [20]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

In [21]:
class Solution:
    def reverseBetween(self, head: Optional[ListNode], left: int, right: int) -> Optional[ListNode]:
        def reverse(l, r):
            if l != r:
                pre, cur = l, l.next
                while pre != r:
                    cur.next = pre
                    pre = cur
                    cur = cur.next
            return r, l
        
        cnt = 0
        dumb = ListNode()
        cur = dumb
        while cur:
            cnt += 1
            if cnt == left: l = cur
            if cnt == right + 1: r = cur
            cur = cur.next
        nr = r.next
        ll, rr = reverse(l.next, r)
        l.next = ll; rr.next = nr
        
        return dumb.next

### 只遍历一次

In [23]:
class Solution:
    def reverseBetween(self, head: ListNode, left: int, right: int) -> ListNode:
        # 设置 dummyNode 是这一类问题的一般做法
        dummy_node = ListNode(-1)
        dummy_node.next = head
        pre = dummy_node
        for _ in range(left - 1):
            pre = pre.next

        cur = pre.next
        for _ in range(right - left):
            nxt = cur.next
            cur.next = nxt.next
            nxt.next = pre.next
            pre.next = nxt
        return dummy_node.next

看不懂

## 415. 字符串相加

In [46]:
class Solution:
    def addStrings(self, num1: str, num2: str) -> str:
        res = []; c = 0
        p1, p2 = len(num1), len(num2)
        while p1 > 0 or p2 > 0:
            n1 = n2 = 0
            if p1 > 0: n1 = int(num1[p1 - 1])
            if p2 > 0: n2 = int(num2[p2 - 1])
            nc, cur = divmod(n1 + n2 + c, 10)
            res.append(str(cur))
            c = nc
            p1 -= 1; p2 -= 1
        if c: res.append("1")
        return "".join(res[::-1])

In [49]:
num1 = "1"; num2 = "999999999999999"

In [50]:
s = Solution()
s.addStrings(num1, num2)

'1000000000000000'

## 142. 环形链表

In [51]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, x):
        self.val = x
        self.next = None

In [53]:
class Solution:
    def detectCycle(self, head: Optional[ListNode]) -> Optional[ListNode]:
        vis = set(); cur = head
        while cur:
            if cur in vis: return cur
            vis.add(cur)
            cur = cur.next
        return

### 可以空间复杂度为O(1)，但是看不懂

## 143. 重排链表

In [1]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

In [59]:
from collections import deque
from typing import Optional
class Solution:
    def reorderList(self, head: Optional[ListNode]) -> None:
        """
        Do not return anything, modify head in-place instead.
        """
        res = deque()
        p = head.next
        while p:
            res.append(p)
            p = p.next
        cur = head
        for i in range(len(res)):
            if i & 1: cur.next = res.popleft()
            else: cur.next = res.pop()
            cur = cur.next    
        cur.next = None

In [60]:
a = [1,2,3,4]
head = modules.constructList(a)

In [61]:
s = Solution()
s.reorderList(head)

In [62]:
while head:
    print(head.val)
    head = head.next

1
4
2
3


## 124. 二叉树中的最大路径和

In [63]:
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

In [66]:
class Solution:
    def maxPathSum(self, root: Optional[TreeNode]) -> int:
        def dfs(node):
            nonlocal ans
            if not node: return 0
            cur = node.val
            left = dfs(node.left)
            right = dfs(node.right)
            # 维护最大路径和的信息
            ans = max(ans, cur + max(left, 0) + max(right, 0))
            # 对于父节点的贡献只能返回两条路径中较大的那一条，否则不符合路径的定义
            return cur + max(max(left, right), 0)
        
        ans = float("-inf")
        dfs(root)
        return ans

## 二叉树的中序遍历

In [68]:
from typing import List
class Solution:
    def inorderTraversal(self, root: Optional[TreeNode]) -> List[int]:
        res = []
        def dfs(node):
            if not node: return
            dfs(node.left)
            res.append(node.val)
            dfs(node.right)
        return res

In [69]:
class Solution:
    def inorderTraversal(self, root: Optional[TreeNode]) -> List[int]:
        res = []
        stk = []
        cur = root
        while stk or cur:
            while cur:
                stk.append(cur)
                cur = cur.left
            cur = stk.pop()
            res.append(cur.val)
            cur = cur.right
        return res

## 19.删除链表的倒数第N个节点

dumb节点的重要性

In [70]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

In [71]:
class Solution:
    def removeNthFromEnd(self, head: Optional[ListNode], n: int) -> Optional[ListNode]:
        p1 = p2 = dumb = ListNode()
        dumb.next = head
        for _ in range(n):
            p2 = p2.next
        while p2.next:
            p1 = p1.next
            p2 = p2.next
        p1.next = p1.next.next
        return dumb.next

## 72. 编辑距离

> 想想二维的问题该如何递推：对于dp[i][j] 的最好情况，一定是从相邻的 dp[i - 1][j], dp[i][j - 1], dp[i - 1][j - 1] 转移过来，分别考虑转移条件与dp如何更新即可。针对此题只需注意在word1[i] == word2[j] 时的特殊情况，其余状态直接取最小再转移即可。

仔细想想其实这题是一个马尔科夫链模型，每一步的最优情况均从上一步的最优情况转移而来。重点是如何才能想到构建二维dp这个模式。

In [110]:
class Solution:
    def minDistance(self, word1: str, word2: str) -> int:
        
        m = len(word1)
        n = len(word2)
        dp = [[0] * (n + 1) for _ in range(m + 1)]
        for i in range(m + 1):
            for j in range(n + 1):
                if i == 0: dp[i][j] = j
                if j == 0: dp[i][j] = i
        
        # print(dp)
        
        for i in range(1, m + 1):
            for j in range(1, n + 1):
                if word1[i - 1] == word2[j - 1]: dp[i][j] = min(dp[i - 1][j], dp[i][j - 1], dp[i - 1][j - 1] - 1) + 1
                else: dp[i][j] = min(dp[i - 1][j], dp[i][j - 1], dp[i - 1][j - 1]) + 1
        
        # print(dp)
        return dp[-1][-1]

In [111]:
word1 = "horse"
word2 = "ros"

In [112]:
s = Solution()
s.minDistance(word1, word2)

3

## 232. 用栈实现队列

In [116]:
class MyQueue:

    def __init__(self):
        self.que = list()
        self.left = 0

    def push(self, x: int) -> None:
        self.que.append(x)

    def pop(self) -> int:
        if not self.empty():
            self.left += 1
            return self.que[self.left - 1]

    def peek(self) -> int:
        if not self.empty():
            return self.que[self.left]

    def empty(self) -> bool:
        return bool(len(self.que) == self.left)

### 双栈

将一个栈当作输入栈，用于压入 push 传入的数据；另一个栈当作输出栈，用于 pop 和 peek 操作。
每次 pop 或 peek 时，若输出栈为空则将输入栈的全部数据依次弹出并压入输出栈，这样输出栈从栈顶往栈底的顺序就是队列从队首往队尾的顺序。

In [117]:
class MyQueue:

    def __init__(self):
        self.stk = list()
        self.que = list()

    def push(self, x: int) -> None:
        self.stk.append(x)

    def pop(self) -> int:
        if not self.que:
            while self.stk:
                self.que.append(self.stk.pop())
        return self.que.pop()
        
    def peek(self) -> int:
        if not self.que:
            while self.stk:
                self.que.append(self.stk.pop())
        return self.que[-1]

    def empty(self) -> bool:
        return not bool(self.stk or self.que)